## Deep Learning Course Project 3: RNN

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

458752/600901 [=====================>........] - ETA: 0scorpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 58
nb sequences: 200285


In [4]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Vectorization...
Build model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()



--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 424s - loss: 1.9809   

----- diversity: 0.2
----- Generating with seed: "sible is becoming extinct. we realize th"
sible is becoming extinct. we realize the ligions of the self as the more the strean the strenges the self an an an an imperience of the liginations and the pressical the strenges of the man in the periated and the still the strenge the strenges and the more the self the strenges and the stander the liginated the strean the stander the more the strenge the strenge the stance of the strenges and the means and the stander the such an an a

----- diversity: 0.5
----- Generating with seed: "sible is becoming extinct. we realize th"
sible is becoming extinct. we realize the self man the more finated than the thing and here in strean the stence and as the actions of the more strether and the intertained and and the reas and the fical and into the refucted t

 operate as an additional incentive and some most powers of the sort of the soul of the street the strength of the sacrifice in the problem of the constitute the thought the man the soul of the sort of the street of the highest the conscience the soul of the sense, and in the philosophers and the street of the morality of the sense, and the stands and the sense of the higher so that is the power and the philosophers in the popular the p

----- diversity: 0.5
----- Generating with seed: " operate as an additional incentive and "
 operate as an additional incentive and perceive prevent fashtory and does not
finally the consider only the lates and makes of many presticiple some man is all the self point even than the more consequently enjoy more of the highest to so the true the passions in the most powers with the taid how doubt, permind of the case the trading the sellce" and the believed to the world, have the sense that is the instrumanily and a strength the 

----- diversity: 1.0
---

C:\mySpace\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 and and evil the engence in the historical even to suffering and not to so sould stalk the substautical inception that it has all this progress of the fullence as a play to the strive and the truth and remain strange, the interrorally completed the the person for will and desires the same problem of the proud and how and consequently the deep problems 

----- diversity: 1.0
----- Generating with seed: "ourselves obliged to give credence to a
"
ourselves obliged to give credence to a
demoblenes that
there
sets
consciousness on
the ridy perros
starty. they are powens
dangrathible to the heart schopenhauer will cease of parting has their more the hesperity of those out oply to so, neversher", through
prefoust, and fleed, the philosopherm, or god much as the soul,
let our obbeal, antither in the in problem of evining
upon the brutely case consure and its wdin nakest, at the lash 

----- diversity: 1.2
----- Generating with seed: "ourselves obliged to give credence to a
"
ourselves obliged

as many philosophers is the works and civilization and conscience he who would be a long most concicing comparivation of the work of still, it without a probably has been man that there is not a terrective and what 

----- diversity: 1.0
----- Generating with seed: "lie the moments of rest, of
not-feeling:"
lie the moments of rest, of
not-feeling:
to ranctly philosophy
still such justice really still--and is
counts it very nogles for the ownspan of nature
ofsesolongs
the
moral power, languagathts in noote what was main want, that can been a grodegness of reference plawe to bake, or artfule as out of it alofed kneared to though theirs do sofningness in the certain others amotimatists of human mighes and onlessity and morality againiteded bu

----- diversity: 1.2
----- Generating with seed: "lie the moments of rest, of
not-feeling:"
lie the moments of rest, of
not-feeling:--wi get the unilotial human--that
it famat locked thoughts their hatieships:
because
hoo
althowncrbitue which, for a

...

themselves in due course, as in the consideration of the proportion of the belief in the stands and an action of the stands of the strength of the strength of the process of the superiority and the strength of the constitute when the process of the strength of the process of the morality and the morality and the properly in the and an action of the morality and more and an another in the strength of the strength of the fact that the 

----- diversity: 0.5
----- Generating with seed: "rt
themselves in due course, as in the c"
rt
themselves in due course, as in the could be a fact of the soul of the depression of the customate more and his himself to the strength and strength of caute and better is always been belief to the highest problem of the greatest the strive nothing is solele spirits and best only the strean constitutes and only by the truth was always have been the suports and creature of such a man who has more confineced so the belief with the abit

----- diversity: 1.0
----- 

not believe, but it does not the self-deceptions and and the most said to the senses of the proporation of the standard of the proporation of the standard of the self-deception of the proporation of the standard of the proporation of the depermed to the serious state of the senses of the senses of the senses of the proporation of the proporation of the serious strength of the proporation of the proporation of the and a stren

----- diversity: 0.5
----- Generating with seed: "on; it does
not believe, but it does not"
on; it does
not believe, but it does not understand and finally the power is the real of the end by serious thing and
easily little and the last superstition of the self-disa; just which has moral of the reality of an all the misunderstandance of his thing contemplation of souls and friends, and to our dream of the primantician or always be the man in the sense, and with his own things. there is the most desires of the truth men who wer

----- diversity: 1.0
----- Generatin

betray in the most such a soul of the soul of the philosophy of the profound to the sublimes and the morality and the subtle of the probaid of the world and despised and some things the former the soul and individual as the soul a some precisely the soul of the subject of the power as the world in the service and an action, and in the probaid and profound and in the same spirit of the probaid and dis

----- diversity: 0.5
----- Generating with seed: ": even today matters are very little
bet"
: even today matters are very little
betray philosophers and individual and the good discovered to be nature of the same man is the law of the morality the strengths, which also, and who will not experience and new which will as the once as a probably in the serves an an
intellectual others of which they have a nature of the work and little asmitter is motive. in itself in the world is a particularies and love to a philosophy of the wor

----- diversity: 1.0
----- Generating with seed: ": even toda

will suffer inwards the sense of the proceed and the truth is a strunged to the proceed of the philosophy of the senses of the constantly always believe the priced to the service and all the power, and and the sense of the moral philosophers and sainted the service to the priced to the service of the still and as the philosophers and action of the senses of the senses of the senses of the depth of the philosophe

----- diversity: 0.5
----- Generating with seed: "king altogether, or they
will suffer inw"
king altogether, or they
will suffer inwards
the priced and appearance of moralities--we knowledge is still themself, when they are not demotion of their example, the morality are not because the conception to the christianity and constantly above logice of the latter that it a philosophers, in a stricted to some action and morality--a subject, and said to a strength of the pain at least the proceeds of the things is at all their princi

----- diversity: 1.0
----- Generating with seed: 

came and the present and state the conscience to the most sound the senses of the most interpreted the soul of the world and such a state the more conscience of the profound to the most such a strength of the most sound to the belief in the most such a sought in the master of the world and struggle and superficial of the most satisfaction of the same something interpreted to a soul of the most self--

----- diversity: 0.5
----- Generating with seed: " grew out of dread or necessity, and
cam"
 grew out of dread or necessity, and
came being with every soul of the welfietic servants and distrust, when the noble other most stands, and and regarded to ascenden out of feelings of body there is the life called the state the common things such development of the experiment, and common and such a mildity, the old strength, but present the state them as it was the master of the light, and the heavens and intellectual as the disciplin

----- diversity: 1.0
----- Generating with seed: " grew out o